In [ ]:
import cv2
from copy import deepcopy
img = cv2.imread(r'D:\Desktop\Data\BUFF\Temp\A-BLUE-STATIC-04_00160.jpg')
temp=deepcopy(img)
img[...,0],img[...,1],img[...,2]=temp[...,2]+50,temp[...,1],temp[...,0]
print(img[0][0])
cv2.imshow("x",img)
cv2.waitKey()
cv2.destroyAllWindows()

In [13]:
import cv2 as cv
import numpy as np

def contrast_brightness_image(src1, a, g):
    h, w, ch = src1.shape # 获取shape的数值，height、width和通道

    # 新建全零图片数组src2，将height、width，类型设置为原图片的通道类型（色素全为零，输出为全黑图片）
    src2 = np.zeros([h, w, ch], src1.dtype)
    dst = cv.addWeighted(src1, a, src2, 1-a, g)
    # cv.namedWindow("con-bri-demo",cv.WINDOW_NORMAL)
    cv.imshow("con-bri-demo", dst)

src = cv.imread(r"D:\Desktop\Data\BUFF\Enhance\BLUE2000+RED400+BRIGHT\A-BLUE-STATIC-04_00160.jpg")
# cv.namedWindow("oldImage",cv.WINDOW_NORMAL)
cv.imshow("oldImage", src)
contrast_brightness_image(src, 1.5, 30) # 第一个1.2为对比度；第二个为亮度数值，越大越亮
cv.waitKey(0)
cv.destroyAllWindows()

In [28]:
import numpy as np
import cv2
#
img = cv.imread(r"D:\Desktop\Data\BUFF\BLUE-2000-FIX\400x2+700+900\A-BLUE-STATIC-04_00160.jpg")

print(img.shape)
cv2.imshow('original',img)
#
# 水平镜像
#flipCode=1 水平翻转;  flipCode= 0 垂直翻转; flipCode=-1 水平垂直 ;
h_flip = cv2.flip(img,flipCode=1)
# cv2.imshow('Flipped Horizontally',h_flip)
 
# 垂直镜像
v_flip = cv2.flip(img, flipCode=0)
# cv2.imshow('Flipped Vertically',v_flip)
 
# 水平垂直镜像
hv_flip = cv2.flip(img, flipCode=-1)
# cv2.imshow('Flipped Horizontalliy-Verticalliy',hv_flip)
 
# 平移
M = np.array([[1,0,-100],[0,1,-12]],dtype=np.float32)
img_change = cv2.warpAffine(img, M, (300,300))
# cv2.imshow("translation", img_change)
 
# 90度旋转 scale = -1 逆时针，scale = 1 顺时针
rows,cols = img.shape[:2]
M = cv2.getRotationMatrix2D((cols/2,rows/2),90,scale = -1)
dst = cv2.warpAffine(img,M,(cols,rows))
# cv2.imshow("90",dst)
 
#45度旋转
rows,cols=img.shape[:2]
M=cv2.getRotationMatrix2D((cols/2,rows/2),45,1)
dst=cv2.warpAffine(img,M,(cols,rows))
cv2.imshow("45",dst)
 
# 缩放 放大
height,width=img.shape[:2]
# res=cv2.resize(img,(2*width,2*height))
# cv2.imshow("large",res)
 
# 缩小 输入的浮点
res2=cv2.resize(img,(int(width/2),int(height/2)))
# cv2.imshow("large",res2)
 
# 仿射变换
point1=np.float32([[50,50],[300,50],[50,200]])
point2=np.float32([[10,100],[300,50],[100,250]])
M=cv2.getAffineTransform(point1,point2)
dst1=cv2.warpAffine(img,M,(cols,rows),borderValue=(255,255,255))
# cv2.imshow("affine transformation",dst1)
cv2.waitKey(0)
cv2.destroyAllWindows()

NameError: name 'cv' is not defined

标签平移放缩

In [28]:
import os
import cv2
import shutil
import numpy as np
# 读取txt
txtDir=r'D:\Desktop\Data\BUFF\Enhance\BLUE2000+RED400+BRIGHT'
txtNames = os.listdir(txtDir)
backDir= os.path.join(txtDir,"..",'Background')
backNames= os.listdir(backDir)
packPaths= [os.path.join(backDir,name) for name in backNames]

for name in txtNames:
    if ('-STATIC-' not in name) or ('.txt' not in name) or ('-BACK-' in name):
        continue
    
    txtPath = os.path.join(txtDir,name)
    newLines=[]
    with open(txtPath,'r') as f:
        for line in f.readlines():
            line=np.array(line.strip().split()[1:],dtype=np.float64).reshape(7,2)*np.array([640,384])
            while True:
                bias=(np.random.random()*0.2)+1
                if True not in (((line[0]-(line[1]*bias*0.5)))<[0]*2) and True not in (((line[0]+line[1]*bias*0.5)>[640,384])):
                    break
            newSize=(line[1]*bias).astype(np.int64)
            newPoint1=line[0]-newSize*0.5
            
            # line[0]=newPoint1
            # line[1]=newSize
            line[0]-=line[1]*0.5
            line[2:]=((line[2:]-line[0])/line[1])*newSize+newPoint1
            # line[0]=newPoint1
            # line[1]=newSize
            # line=line.astype(np.int64)
            # line=np.array(line.strip().split()[1:],dtype=np.float32)
            # print(txtPath)
            # print(line)
            img = cv2.imread(txtPath.replace('.txt','.jpg'))
            randomint=np.random.randint(len(packPaths))
            background = cv2.imread(packPaths[randomint])

            while True:
                bias=np.array([np.random.randint(-320,320),np.random.randint(-192,192)])
                # bias=np.array([0,0])
                if (True not in (newPoint1+bias <[0]*2)) and (True not in (newPoint1+newSize+bias >[640,384])):
                    break
            backline=line.copy()
            backline[0]=newPoint1
            backline[1]=newSize
            
            backline+=bias
            backline[1]-=bias
            
            backlineInt=backline[:2].astype(np.int64)
            lineInt=line[:2].astype(np.int64)
            
            background[backlineInt[0][1]:backlineInt[0][1]+backlineInt[1][1],backlineInt[0][0]:backlineInt[0][0]+backlineInt[1][0]]=\
                    cv2.resize(img[lineInt[0][1]:lineInt[0][1]+lineInt[1][1],lineInt[0][0]:lineInt[0][0]+lineInt[1][0]],newSize)
                    
            backline[0]+=backline[1]/2
            newLines.append(" ".join(str(i) for i in [0]+sum((backline/[640,384]).tolist(),[])))
            
    newTxtPath=txtPath.replace('-STATIC-','-STATIC-BACK-')
    with open(newTxtPath,'w') as f:
        for newLine in newLines:
            f.write(newLine+'\n')
    cv2.imwrite(newTxtPath.replace('.txt','.jpg'),background)        
    # shutil.copyfile(txtPath.replace('.txt','.jpg'),newTxtPath.replace('.txt','.jpg'))


In [56]:
import numpy as np
# np.random.randint(1)
# False not in (np.array([1,2])>[0]*2)
(np.random.random()*0.4)+0.8

1.1782222837766527

亮度增强

In [1]:
import os
import cv2
import shutil
import numpy as np

def contrast_brightness_image(src1, a, g):
    h, w, ch = src1.shape # 获取shape的数值，height、width和通道

    # 新建全零图片数组src2，将height、width，类型设置为原图片的通道类型（色素全为零，输出为全黑图片）
    src2 = np.zeros([h, w, ch], src1.dtype)
    dst = cv2.addWeighted(src1, a, src2, 1-a, g)
    # cv.namedWindow("con-bri-demo",cv.WINDOW_NORMAL)
    # cv2.imshow("con-bri-demo", dst)
    return dst

def generateBrightImage(txtDir):
    names = os.listdir(txtDir)
    for name in names:
        if ('-STATIC-' not in name) or ('.txt' not in name) or ('-STATIC-BRIGHT-' in name):
            continue

        txtPath=os.path.join(txtDir,name)
        imgPath=txtPath.replace('.txt','.jpg')
        brightTxtPath=txtPath.replace('-STATIC-','-STATIC-BRIGHT-')
        brightImagePath=imgPath.replace('-STATIC-','-STATIC-BRIGHT-')
        
        img = cv2.imread(imgPath)
        img = contrast_brightness_image(img,1.1,-100)
        
        cv2.imwrite(brightImagePath,img)
        shutil.copyfile(txtPath,brightTxtPath)
    #     cv2.imshow('img',img)
    #     cv2.waitKey(1)
    # cv2.destroyAllWindows()

generateBrightImage(r'D:\Desktop\Data\BUFF\Enhance\BLUE2000+RED400+BRIGHT')

背景切换

In [6]:
import os
import cv2
import shutil
import numpy as np

def generateNewBackgroundImage(txtDir):
    BGDir=os.join(txtDir,"..",'Background')
    BGNames=os.listdir(BGDir)
    [os.path.join(BGDir,name) for name in BGNames]
    names = os.listdir(txtDir)
    for name in names:
        if ('-STATIC-' not in name) or ('.txt' not in name) or ('-STATIC-BRIGHT-' in name):
            continue
        
        txtPath=os.path.join(txtDir,name)
        imgPath=txtPath.replace('.txt','.jpg')
        newBGTxtPath=txtPath.replace('-STATIC-','-STATIC-BRIGHT-')
        newBGImagePath=imgPath.replace('-STATIC-','-STATIC-BRIGHT-')
        
        img = cv2.imread(imgPath)
        img = contrast_brightness_image(img,1.1,-100)
        
        # cv2.imwrite(newBackgroundImagePath,img)
        shutil.copyfile(txtPath,newBGTxtPath)
        cv2.imshow('img',img)
        key=cv2.waitKey(1)
        if key == 27:
            break
    cv2.destroyAllWindows()

generateNewBackgroundImage(r'D:\Desktop\Data\BUFF\Enhance\BLUE2000+RED400+BRIGHT')

In [3]:
import cv2
import numpy as np
# img=cv2.imread(r'D:\Desktop\Data\BUFF\Enhance\DYNAMIC-700\0_00024.jpg', cv2.IMREAD_COLOR)    # 打开文件
cap = cv2.VideoCapture(r'E:\LanyeSaint\buff\2108 00_10_36-00_10_50.mp4')
while True:
    ret,img=cap.read()
    if ret:
        img=img[...,::-1]
        # 通过cv2.cvtColor把图像从BGR转换到HSV
        img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

        # H空间中，绿色比黄色的值高一点，所以给每个像素+15，黄色的树叶就会变绿
        turn_green_hsv = img_hsv.copy().astype(np.int16)
        turn_green_hsv[:, :, 0] = (turn_green_hsv[:, :, 0]-10).clip(min=0) % 180
        turn_green_hsv=turn_green_hsv.astype(np.uint8)
        turn_green_img = cv2.cvtColor(turn_green_hsv, cv2.COLOR_HSV2BGR)
        cv2.namedWindow('image',cv2.WINDOW_NORMAL)
        cv2.imshow('image',turn_green_img)
        #cv2.imwrite('./out/turn_green.jpg', turn_green_img)
        key=cv2.waitKey(1)
        if key == 27:
            break
    else:
        break
# 减小饱和度会让图像损失鲜艳，变得更灰
# colorless_hsv = img_hsv.copy()
# colorless_hsv[:, :, 1] = 0.5 * colorless_hsv[:, :, 1]
# colorless_img = cv2.cvtColor(colorless_hsv, cv2.COLOR_HSV2BGR)
# cv2.imshow('image',colorless_img)
# cv2.waitKey(0)
#cv2.imwrite('./out/colorless.jpg', colorless_img)

# 调整明度全为最大，相当于只在意颜色种类，与光线无关了
# darker_hsv = img_hsv.copy()
# darker_hsv[:, :, 2] =255 #0.5 * darker_hsv[:, :, 2]
# darker_img = cv2.cvtColor(darker_hsv, cv2.COLOR_HSV2BGR)
# cv2.imshow('image',darker_img)

# cv2.waitKey(0)
cv2.destroyAllWindows()
#cv2.imwrite('./out/darker.jpg', darker_img)